In [132]:
import numpy as np
import pandas as pd

In [133]:
df_customer=pd.read_csv('train/df_Customers.csv')
df_orderitems=pd.read_csv('train/df_OrderItems.csv')
df_order=pd.read_csv('train/df_Orders.csv')
df_payments=pd.read_csv('train/df_Payments.csv')
df_products=pd.read_csv('train/df_Products.csv')

df=pd.merge(df_order,df_orderitems, on=['order_id'],how='inner')
df=pd.merge(df,df_customer, on=['customer_id'],how='inner')
df=pd.merge(df,df_payments, on=['order_id'],how='inner')
df_products = df_products.drop_duplicates(subset=['product_id'])
df_products=df_products.reset_index(drop=True)
df_products.info()
df=pd.merge(df,df_products, on=['product_id'],how='inner')
df['order_estimated_delivery_date'] = pd.to_datetime(df['order_estimated_delivery_date'])
df['order_delivered_timestamp'] = pd.to_datetime(df['order_delivered_timestamp'])
df['order_delivered_timestamp'] = df['order_delivered_timestamp'].dt.strftime('%Y-%m-%d')
df_tes=df.dropna()
df_tes['order_delivered_timestamp'] = pd.to_datetime(df_tes['order_delivered_timestamp'])
df_tes['is_late'] = df_tes.apply(lambda row: 1 if row['order_delivered_timestamp'] > row['order_estimated_delivery_date'] else 0, axis=1)
df_tes['late'] = df_tes.apply(lambda row: (row['order_estimated_delivery_date'] - row['order_delivered_timestamp']).days, axis=1)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27451 entries, 0 to 27450
Data columns (total 6 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   product_id             27451 non-null  object 
 1   product_category_name  27310 non-null  object 
 2   product_weight_g       27449 non-null  float64
 3   product_length_cm      27449 non-null  float64
 4   product_height_cm      27449 non-null  float64
 5   product_width_cm       27449 non-null  float64
dtypes: float64(4), object(2)
memory usage: 1.3+ MB


C:\Users\justi\AppData\Local\Temp\ipykernel_12044\83762704.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_tes['order_delivered_timestamp'] = pd.to_datetime(df_tes['order_delivered_timestamp'])
C:\Users\justi\AppData\Local\Temp\ipykernel_12044\83762704.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_tes['is_late'] = df_tes.apply(lambda row: 1 if row['order_delivered_timestamp'] > row['order_estimated_delivery_date'] else 0, axis=1)
C:\Users\justi\AppData\Local\Temp\ipykernel_12044\83762704.p

In [134]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.utils import resample


# # Prepare the data
# X = df_tes[["product_weight_g", "product_length_cm", "product_height_cm", "product_width_cm"]]
# y = df_tes["is_late"]

# # Split the data into training and test sets
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

# # Create a logistic regression model
# # model = LogisticRegression()

# clf = xgb.XGBClassifier(
#     objective='binary:logistic',  # For binary classification
#     n_estimators=100,  # Number of boosting rounds (trees)
#     max_depth=3,  # Maximum depth of each tree
#     learning_rate=0.1  # Step size shrinkage
# )

# clf.fit(X_train, y_train)

In [135]:
n_samples_majority = len(df_tes[df_tes['is_late'] == 0])
df_late = resample(df_tes[df_tes['is_late'] == 1],
                   replace=True,
                   n_samples=n_samples_majority,
                   random_state=42)

# Combine the resampled minority class with the majority class
df_balanced = pd.concat([df_late, df_tes[df_tes['is_late'] == 0]])

# Split the balanced data into training and test sets
X = df_balanced[["product_weight_g", "product_length_cm", "product_height_cm", "product_width_cm"]]
y = df_balanced["is_late"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create an XGBoost classifier
clf = xgb.XGBClassifier(
    objective='binary:logistic',  # For binary classification
    n_estimators=500,  # Number of boosting rounds (trees)
    max_depth=100,  # Maximum depth of each tree
    learning_rate=0.1  # Step size shrinkage
    # eta=0.1
)

# param_grid = {
#     'max_depth': [None, 10, 20, 30],  # Contoh nilai untuk max_depth
#     'min_samples_split': [2, 5, 10],  # Contoh nilai untuk min_samples_split
#     'min_samples_leaf': [1, 2, 4]  # Contoh nilai untuk min_samples_leaf
# }



# from sklearn.tree import DecisionTreeClassifier
# clf = DecisionTreeClassifier()
# # clf = GridSearchCV(estimator=grid_search, param_grid=param_grid, cv=3, scoring='accuracy')


# Train the XGBoost model
clf.fit(X_train, y_train)

# Make predictions on the test data
y_pred = clf.predict(X_test)

# Evaluate the model's performance
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

# You can also print a classification report and confusion matrix for more details
print(classification_report(y_test, y_pred))
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

Accuracy: 0.797534725416245
              precision    recall  f1-score   support

           0       0.86      0.72      0.78     16387
           1       0.75      0.88      0.81     16226

    accuracy                           0.80     32613
   macro avg       0.81      0.80      0.80     32613
weighted avg       0.81      0.80      0.80     32613

Confusion Matrix:
[[11746  4641]
 [ 1962 14264]]


In [136]:
# from sklearn.model_selection import GridSearchCV

# # Define the hyperparameter grid to search over
# param_grid = {
#     'max_depth': [None, 10, 20, 30],
#     'min_samples_split': [2, 5, 10],
#     'min_samples_leaf': [1, 2, 4],
#     'criterion': ['gini', 'entropy'],
#     'splitter': ['best', 'random'],
#     'max_features': [None, 'sqrt', 'log2'],
#     'class_weight': [None, 'balanced']
# }

# # Create a Decision Tree Classifier
# clf = DecisionTreeClassifier()

# # Create a GridSearchCV object to search for the best hyperparameters
# grid_search = GridSearchCV(estimator=clf, param_grid=param_grid, cv=3, scoring='accuracy')

# # Fit the GridSearchCV object to your training data
# grid_search.fit(X_train, y_train)

# # Get the best hyperparameters from the search
# best_params = grid_search.best_params_
# print("Best Hyperparameters:", best_params)

# # Use the best hyperparameters to create the final model
# best_clf = DecisionTreeClassifier(**best_params)

# # Train the final model on your training data
# best_clf.fit(X_train, y_train)

# # Make predictions and evaluate the final model
# y_pred = best_clf.predict(X_test)
# accuracy = accuracy_score(y_test, y_pred)
# print("Accuracy:", accuracy)

# # You can also print a classification report and confusion matrix for more details
# print(classification_report(y_test, y_pred))
# print("Confusion Matrix:")
# print(confusion_matrix(y_test, y_pred))


In [137]:
df_customer=pd.read_csv('test/df_Customers.csv')
df_orderitems=pd.read_csv('test/df_OrderItems.csv')
df_order=pd.read_csv('test/df_Orders.csv')
df_payments=pd.read_csv('test/df_Payments.csv')
df_products=pd.read_csv('test/df_Products.csv')

df=pd.merge(df_order,df_orderitems, on=['order_id'],how='inner')
df=pd.merge(df,df_customer, on=['customer_id'],how='inner')
df=pd.merge(df,df_payments, on=['order_id'],how='inner')
df_products = df_products.drop_duplicates(subset=['product_id'])
df_products=df_products.reset_index(drop=True)
df_products.info()
df=pd.merge(df,df_products, on=['product_id'],how='inner')
# df['order_estimated_delivery_date'] = pd.to_datetime(df['order_estimated_delivery_date'])
# df['order_delivered_timestamp'] = pd.to_datetime(df['order_delivered_timestamp'])
# df['order_delivered_timestamp'] = df['order_delivered_timestamp'].dt.strftime('%Y-%m-%d')
# df_tes=df.dropna()
# df_tes['order_delivered_timestamp'] = pd.to_datetime(df_tes['order_delivered_timestamp'])
# df_tes['is_late'] = df_tes.apply(lambda row: 1 if row['order_delivered_timestamp'] > row['order_estimated_delivery_date'] else 0, axis=1)
# df_tes['late'] = df_tes.apply(lambda row: (row['order_estimated_delivery_date'] - row['order_delivered_timestamp']).days, axis=1)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16604 entries, 0 to 16603
Data columns (total 6 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   product_id             16604 non-null  object 
 1   product_category_name  16514 non-null  object 
 2   product_weight_g       16603 non-null  float64
 3   product_length_cm      16603 non-null  float64
 4   product_height_cm      16603 non-null  float64
 5   product_width_cm       16603 non-null  float64
dtypes: float64(4), object(2)
memory usage: 778.4+ KB


In [139]:
X = df[["product_weight_g", "product_length_cm", "product_height_cm", "product_width_cm"]]
df['is_late']= clf.predict(X)
df=df[['order_id','is_late']]
df

,order_id,is_late
0,u6rPMRAYIGig,0
1,W883pN454zIH,0
2,S7zJd1ziBFGA,0
3,ohY8f4FEbX19,1
4,hI8Q4fQlN2Pu,1
...,...,...
38274,C6RN6xwcBVTe,0
38275,tkxVd3HFludF,0
38276,DI3Jz2puVexJ,0
38277,kmViZI3t6fUT,0


In [141]:
jumlah_is_late_1 = df['is_late'].sum()

# Cetak jumlahnya
print("Jumlah baris yang bernilai 1 pada kolom 'is_late':", jumlah_is_late_1)

Jumlah baris yang bernilai 1 pada kolom 'is_late': 11212


In [ ]:
df.to_csv('justin-xgboost.csv', index=False)